In [4]:
import os
import shutil
import filecmp
import logging
from tqdm.auto import tqdm

# Configure logging
logging.basicConfig(level=logging.ERROR,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    filename='file_transfer_log.txt')

# Set your source and destination directories here
source_directory = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Shared - Baptiste Pradel/4i/Live-cell to 4i for Nathan'
destination_directory = '/mnt/DATA3/BPP0050'

# Ensure both directories exist
if not os.path.exists(source_directory):
    print(f"Error: Source directory '{source_directory}' does not exist.")
    exit()
if not os.path.exists(destination_directory):
    print(f"Error: Destination directory '{destination_directory}' does not exist. Creating it.")
    os.makedirs(destination_directory)

# Get the total number of files to copy
total_files = 0
for root, _, files in os.walk(source_directory):
    total_files += len(files)

# Use os.walk to traverse the source directory
copied_files = 0
with tqdm(total=total_files, unit='file') as pbar:
    for root, _, files in os.walk(source_directory):
        for file in files:
            source_file_path = os.path.join(root, file)
            dest_file_path = os.path.join(destination_directory, os.path.relpath(source_file_path, source_directory))

            if not os.path.exists(dest_file_path):
                # print(f"Copying missing file: '{source_file_path}' to '{dest_file_path}'")
                os.makedirs(os.path.dirname(dest_file_path), exist_ok=True)
                try:
                    shutil.copy2(source_file_path, dest_file_path)
                    copied_files += 1
                    pbar.update(1)
                except Exception as e:
                    error_message = f"Error copying file: '{source_file_path}': {e}"
                    print(error_message)
                    logging.error(error_message)
            elif not filecmp.cmp(source_file_path, dest_file_path, shallow=False):
                # print(f"Copying changed file: '{source_file_path}' to '{dest_file_path}'")
                try:
                    shutil.copy2(source_file_path, dest_file_path)
                    copied_files += 1
                    pbar.update(1)
                except Exception as e:
                    error_message = f"Error copying file: '{source_file_path}': {e}"
                    print(error_message)
                    logging.error(error_message)
            else:
                # print(f"Skipping identical file: '{source_file_path}'")
                pbar.update(1) #still update even if skipped.
# print(f"Copied {copied_files} files.")


  0%|          | 0/790328 [00:00<?, ?file/s]